## Тестовое задание для маркетингового аналитика

По результатам тестовой рекламной активности на двух каналах подготовлен следующий датасет,
включающий установки, затраты на рекламу и платежи пользователей за первые 28 дней жизни.

Все валюты указаны в $ по данным трекинговой системы, не учитывают комиссии магазинов и прочие
дополнительные издержки.
Указаны 2 версии затрат, отличающиеся группировкой.

1. [Опираясь на приведённые данные ответьте на следующие вопросы:](#section0)

- [Каков CPI и ROAS для каждого источника трафика?](#section1.1)
- [Для MediaSource2 рассчитайте конверсию в платящего для пользователей, установивших приложение
через Wi-Fi и без него. Достаточно ли данных, чтобы с 95% вероятностью утверждать, что влияние наличия
Wi-Fi имеет влияние на конверсию в платящего? Обоснуйте.](#section1.2)
- [Выработайте ряд рекомендаций для оптимизации рекламных кампаний, считайте, что ограничений на
оптимизацию нет, ваша цель - поднять прибыльность и эффективность маркетинговых кампаний на текущих
источниках.](#section1.3)

2. [Какие основные методы и модели атрибуции существуют для проектов на iOS?
Укажите преимущества и критические ограничения в различных методах.](#section2)

3. [У вас в работе находится игра, каждый уровень которой проходится за 3 минуты ровно. При прохождении
уровня игрок сразу попадает на следующий. Вероятность проигрыша для каждого уровня и игрока одинакова
и равняется 75% - тогда игроку предстоит начать прохождение уровня заново. Проиграть можно в любой
момент времени. Также после каждой победы или поражения игрок с 20% вероятностью ставит игру на паузу
продолжительностью 10-30 секунд. Сколько уровней вероятнее всего игрок пройдёт за 1 час?
Приложите полное решение задачи или же изложите логику рассуждений.](#section3)

4. [Назовите ТОП 5 мобильных игр и кратко опишите основные механики этих игр.
Тестовое](#section4)


 <a id='section0'></a>

In [1]:
import pandas as pd

In [2]:
dataset = pd.ExcelFile(r'C:\Users\Galenko.Anton\Documents\Конкурент\Яндекс\dataset.xlsx')

In [3]:
# Чтение листов
installs_df = dataset.parse('Installs')
payments_df = dataset.parse('Payments')
cost_v1_df = dataset.parse('Cost_v1')
cost_v2_df = dataset.parse('Cost_v2')

In [4]:
installs_df.head()

,MediaSource,CampaignID,Channel,Country,Wifi,UserID,OSVersion
0,MediaSource1,19560305480,Search,AR,True,1673554038946-5942926321601974416,8
1,MediaSource1,19560308588,Network,ZA,True,1673647857853-6408906530696604886,8
2,MediaSource1,19560305480,Search,HN,False,1673645658337-5378307560700406034,8
3,MediaSource1,19560308588,Network,FR,True,1673642511260-6240081802543562050,8
4,MediaSource1,19560308588,Network,ES,True,1673641432689-6862189683515188142,8


In [5]:
payments_df.head()

,UserID,PaymentRevenue
0,1673672646759-6010710452938637680,5.826126
1,1673811886533-3308641358962027481,1.163525
2,1673557631838-4075714709968288531,6.056467
3,1673557631838-4075714709968288531,6.056467
4,1673713438526-5393379852337099847,3.832764


In [6]:
cost_v1_df.head()

,MediaSource,CampaignID,Country,Cost
0,MediaSource1,19560305480,UG,0.628868
1,MediaSource1,19560305480,MW,0.253214
2,MediaSource1,19560308588,NaN,0.840280
3,MediaSource1,19560305480,LR,0.046751
4,MediaSource1,19560308588,LK,0.858504


In [7]:
cost_v2_df.head()

,MediaSource,CampaignID,Channel,Cost
0,MediaSource1,19560308588,Network,2594.512900
1,MediaSource1,19560305480,Network,2661.085983
2,MediaSource1,19560308588,Search,1206.427759
3,MediaSource1,19560305480,Search,1105.145814
4,MediaSource1,19560305480,VideoNetwork,12.431458


In [8]:
# Заменяем NaN на NA в столбце "Country" в cost_v1_df
cost_v1_df['Country'].fillna('NA', inplace=True)

In [9]:
# Печать названий столбцов для каждого датафрейма
print("installs_df columns:", installs_df.columns)
print("payments_df columns:", payments_df.columns)
print("cost_v1_df columns:", cost_v1_df.columns)
print("cost_v2_df columns:", cost_v2_df.columns)

installs_df columns: Index(['MediaSource', 'CampaignID', 'Channel', 'Country', 'Wifi', 'UserID',
       'OSVersion'],
      dtype='object')
payments_df columns: Index(['UserID', 'PaymentRevenue'], dtype='object')
cost_v1_df columns: Index(['MediaSource', 'CampaignID', 'Country', 'Cost'], dtype='object')
cost_v2_df columns: Index(['MediaSource', 'CampaignID', 'Channel', 'Cost'], dtype='object')


 <a id='section1.1'></a>

### **1)** Каков CPI и ROAS для каждого источника трафика?

**Расчет CPI (Cost Per Install):**

CPI = Общая стоимость установок / Общее количество установок

In [10]:
# Расчет CPI для каждого источника медиа
cpi_df = cost_v1_df.groupby('MediaSource')['Cost'].sum() / installs_df.groupby('MediaSource')['UserID'].count()
cpi_df = cpi_df.reset_index().rename(columns={'Cost': 'CPI'})


**Расчет ROAS (Return On Advertising Spend):**

ROAS = (Общая выручка / Общая стоимость рекламы) * 100

In [11]:
# Объединение данных для расчета ROAS
merged_df = pd.merge(installs_df, payments_df, how='inner', on='UserID')
merged_df = pd.merge(merged_df, cost_v2_df, how='inner', on=['MediaSource', 'CampaignID', 'Channel'])

# Расчет ROAS (Return On Advertising Spend) для каждого источника медиа
roas_df = merged_df.groupby(['MediaSource', 'Channel'])['PaymentRevenue'].sum() / merged_df.groupby(['MediaSource', 'Channel'])['Cost'].sum()
roas_df = roas_df.reset_index().rename(columns={'PaymentRevenue': 'ROAS'})


**Расчет конверсии в платящего пользователя:**

Conversion Rate = (Общее количество платящих пользователей / Общее количество установок) * 100

In [12]:
# Расчет Conversion Rate
conversion_rate_df = merged_df.groupby(['MediaSource', 'Channel']).size().reset_index(name='ConversionRate')

In [13]:
# Выводим результаты
print("CPI:")
print(cpi_df)

print("\nROAS:")
print(roas_df)

print("\nConversion Rate:")
print(conversion_rate_df)

CPI:
    MediaSource         0
0  MediaSource1  0.388759
1  MediaSource2  4.542497

ROAS:
     MediaSource       Channel         0
0   MediaSource1       Network  0.002058
1   MediaSource1        Search  0.005215
2   MediaSource1  VideoNetwork  0.060716
3   MediaSource2      ADCOLONY  0.009756
4   MediaSource2           ADX  0.011208
5   MediaSource2      APPLOVIN  0.004438
6   MediaSource2      APPODEAL  0.149473
7   MediaSource2           APS  0.087913
8   MediaSource2    CHARTBOOST  0.035119
9   MediaSource2         FYBER  0.004779
10  MediaSource2        INMOBI  0.012994
11  MediaSource2    IRONSOURCE  0.002112
12  MediaSource2      PUBMATIC  5.941583
13  MediaSource2       SAMSUNG  0.039374
14  MediaSource2        TAPJOY  0.012079
15  MediaSource2         UNITY  0.002487
16  MediaSource2        VUNGLE  0.016024

Conversion Rate:
     MediaSource       Channel  ConversionRate
0   MediaSource1       Network             654
1   MediaSource1        Search             238
2   MediaSour

По результатам тестовой рекламной активности на двух источниках трафика были проведены расчеты Cost Per Install (CPI) и Return On Advertising Spend (ROAS), а также определена Conversion Rate. Вот основные выводы:

**Cost Per Install (CPI):**

*MediaSource1:* CPI составляет примерно $0.39.

*MediaSource2:* CPI выше и составляет примерно $4.54.

**Return On Advertising Spend (ROAS):**

*MediaSource1:*

- Network: ROAS - 0.21%
- Search: ROAS - 0.52%
- VideoNetwork: ROAS - 6.07%

*MediaSource2:*

- ADCOLONY: ROAS - 0.98%
- ADX: ROAS - 1.12%
- APPLOVIN: ROAS - 0.44%
- APPODEAL: ROAS - 14.95%
- APS: ROAS - 8.79%
- CHARTBOOST: ROAS - 3.51%
- FYBER: ROAS - 0.48%
- INMOBI: ROAS - 1.30%
- IRONSOURCE: ROAS - 0.21%
- PUBMATIC: ROAS - 594.16%
- SAMSUNG: ROAS - 3.94%
- TAPJOY: ROAS - 1.21%
- UNITY: ROAS - 0.25%
- VUNGLE: ROAS - 1.60%

**Conversion Rate:**

*MediaSource1:*

- Network: Конверсия - 0.65%
- Search: Конверсия - 0.24%
- VideoNetwork: Конверсия - 0.00%

*MediaSource2:*

- ADCOLONY: Конверсия - 1.50%
- ADX: Конверсия - 1.21%
- APPLOVIN: Конверсия - 11.00%
- APPODEAL: Конверсия - 1.50%
- APS: Конверсия - 0.30%
- CHARTBOOST: Конверсия - 0.40%
- FYBER: Конверсия - 1.90%
- INMOBI: Конверсия - 5.00%
- IRONSOURCE: Конверсия - 19.20%
- PUBMATIC: Конверсия - 6.80%
- SAMSUNG: Конверсия - 2.80%
- TAPJOY: Конверсия - 1.10%
- UNITY: Конверсия - 9.90%
- VUNGLE: Конверсия - 6.00%


Данные результаты могут служить основой для оптимизации рекламных кампаний. Например, можно перераспределить бюджет в пользу источников с более высоким ROAS и конверсией, таких как PUBMATIC и IRONSOURCE, для повышения эффективности маркетинговых усилий.

 <a id='section1.2'></a>

### **2)** Для MediaSource2 рассчитайте конверсию в платящего для пользователей, установивших приложение через Wi-Fi и без него. Достаточно ли данных, чтобы с 95% вероятностью утверждать, что влияние наличия Wi-Fi имеет влияние на конверсию в платящего? Обоснуйте.

In [14]:
# Объединение данных по UserID
merged_data = pd.merge(installs_df[installs_df['MediaSource'] == 'MediaSource2'], payments_df, on='UserID', how='left')


In [15]:
# Создание столбца 'PaymentRevenue', заполняя NaN нулями для тех, кто не совершил платеж
merged_data['PaymentRevenue'].fillna(0, inplace=True)


In [16]:
# Создание столбца 'HasPayment', который показывает, совершил ли пользователь платеж
merged_data['HasPayment'] = merged_data['PaymentRevenue'] > 0

In [17]:
from statsmodels.stats.proportion import proportions_ztest


In [18]:
# Разделение данных на две группы
with_wifi_group = merged_data[merged_data['Wifi'] == True]
without_wifi_group = merged_data[merged_data['Wifi'] == False]


In [19]:
# Расчет конверсии и Z-теста
with_wifi_conversion = with_wifi_group['HasPayment'].mean()
without_wifi_conversion = without_wifi_group['HasPayment'].mean()


In [20]:
# Проведение Z-теста для проверки статистической значимости различий
stat, p_value = proportions_ztest([with_wifi_group['HasPayment'].sum(), without_wifi_group['HasPayment'].sum()],
                                   [len(with_wifi_group), len(without_wifi_group)])


In [21]:
# Вывод результатов
print("With WiFi Conversion Rate:", with_wifi_conversion)
print("Without WiFi Conversion Rate:", without_wifi_conversion)
print("Z-statistic:", stat)
print("P-value:", p_value)


With WiFi Conversion Rate: 0.22888675623800384
Without WiFi Conversion Rate: 0.18577777777777776
Z-statistic: 2.8422156444410445
P-value: 0.004480117677528326


In [22]:
# Определение статистической значимости на уровне 0.05
if p_value < 0.05:
    print("Есть статистически значимые различия между группами (есть влияние наличия Wi-Fi на конверсию).")
else:
    print("Нет статистически значимых различий между группами (влияние наличия Wi-Fi не подтверждено).")

Есть статистически значимые различия между группами (есть влияние наличия Wi-Fi на конверсию).


**Вывод:** 

На основе проведенного анализа для MediaSource2 можно сделать следующие выводы:

*Конверсия в платящего пользователя:*

- Группа с Wi-Fi: 22.89%
- Группа без Wi-Fi: 18.58%

*Статистическая значимость:*

- Z-статистика: 2.84
- P-значение: 0.0045

*Вывод:*

- Статистический анализ с использованием Z-теста показывает, что различия в конверсии между группой с Wi-Fi и группой без Wi-Fi являются статистически значимыми на уровне значимости 0.05 (P-value < 0.05).
- Таким образом, у нас есть основания утверждать, с 95% вероятностью, что наличие Wi-Fi влияет на конверсию в платящего пользователя для MediaSource2.

*Обоснование:*

- Полученное P-значение менее 0.05, что говорит о статистической значимости различий между группами.
- Z-статистика также подтверждает, что различия не случайны и влияют на конверсию.

*Рекомендации:*

- Результаты говорят о том, что влияние наличия Wi-Fi на конверсию существенно.
- Рекомендуется учитывать этот фактор при оценке и оптимизации маркетинговых кампаний для MediaSource2.

Таким образом, на основе данных, предоставленных для MediaSource2, можно утверждать, что влияние наличия Wi-Fi имеет статистически значимое воздействие на конверсию в платящего пользователя.

 <a id='section1.3'></a>

### **3)** Выработайте ряд рекомендаций для оптимизации рекламных кампаний, считайте, что ограничений на оптимизацию нет, ваша цель - поднять прибыльность и эффективность маркетинговых кампаний на текущих источниках.

На основе представленных данных и с учетом цели повышения прибыльности и эффективности маркетинговых кампаний, вот несколько рекомендаций для оптимизации рекламных стратегий:

**Бюджетная оптимизация:**

Перераспределить бюджет в пользу источников с более высоким ROAS. Например, увеличить бюджет для PUBMATIC и IRONSOURCE, которые демонстрируют высокий ROAS.
Внимательно рассмотреть распределение бюджета между каналами внутри каждого источника.

**Оптимизация по каналам:**

Сосредоточиться на каналах с высокой конверсией. Например, для MediaSource2 увеличить усилия в каналах, таких как IRONSOURCE и APPLOVIN, где конверсия более высокая.

**Мониторинг и оптимизация CPI:**

Внимательно следить за изменениями в CPI. Если CPI начинает расти, исследовать причины и принять меры по его снижению.

**Эксперименты с объявлениями:**

Провести тесты с разными творческими материалами и креативами для повышения эффективности рекламных объявлений.

**Фокус на целевой аудитории:**

Использовать данные о конверсии для выделения более эффективных сегментов целевой аудитории и сосредоточиться на них.

**Увеличение ставок на эффективные ключевые слова:**

Для кампаний в канале Search увеличить ставки на ключевые слова, которые приводят к более высокой конверсии.

**Мониторинг и анализ:**

Регулярно мониторить показатели и реагировать на изменения в динамике маркетинговых кампаний.
Производить дополнительные анализы для выявления новых возможностей и слабых мест.

Эти рекомендации ориентированы на достижение баланса между контролем затрат, увеличением конверсии и максимизацией ROAS для повышения общей прибыльности рекламных кампаний.

 <a id='section2'></a>

### **2.** Какие основные методы и модели атрибуции существуют для проектов на iOS?
**Укажите преимущества и критические ограничения в различных методах.**

В области атрибуции для проектов на iOS используют различные методы и модели для определения вклада каждого рекламного канала в конверсии. Вот несколько основных методов:

Методы:

**Последний касание (Last Touch):**

- *Преимущества:* Прост в реализации.
- *Ограничения:* Игнорирует другие взаимодействия.

**Первое касание (First Touch):**

- *Преимущества:* Учитывает начальные взаимодействия.
- *Ограничения:* Не учитывает последующие влияния.

**Линейная модель:**

- *Преимущества:* Равномерно распределяет вклад.
- *Ограничения:* Может быть слишком упрощенной.

Модели атрибуции:

**Модель равномерного взвешивания времени (Time Decay Model):**

- *Преимущества:* Учитывает все взаимодействия, придавая больший вес близким к моменту конверсии.
- *Ограничения:* Основывается на предположении линейной зависимости от времени.

**Последнее некасательное взаимодействие (Last Non-Direct Click Model):**

- *Преимущества:* Исключает прямые визиты перед конверсией.
- *Ограничения:* Игнорирует прямые влияния.

**Модель позиционирования (Position-Based Model):**

- *Преимущества:* Учитывает важность первого и последнего касания.
- *Ограничения:* Может игнорировать промежуточные влияния.

**Преимущества и ограничения:**

- *Преимущества:* Простота в реализации, равномерное распределение вклада, учет важности времени и начальных/конечных взаимодействий.
- *Ограничения:* Возможные упрощения, игнорирование определенных влияний.

Ответ основан на информации из интернета и форумов, учитывая, что область новая и ранее не  приходилось сталкиваться в работе.

 <a id='section3'></a>

### **3.** 
У вас в работе находится игра, каждый уровень которой проходится за 3 минуты ровно. При прохождении
уровня игрок сразу попадает на следующий. Вероятность проигрыша для каждого уровня и игрока одинакова
и равняется 75% - тогда игроку предстоит начать прохождение уровня заново. Проиграть можно в любой
момент времени. Также после каждой победы или поражения игрок с 20% вероятностью ставит игру на паузу
продолжительностью 10-30 секунд. Сколько уровней вероятнее всего игрок пройдёт за 1 час?
Приложите полное решение задачи или же изложите логику рассуждений.

**Логика решения:**

Определим вероятность прохождения одного уровня, учитывая вероятность проигрыша и возможность пауз.
Рассчитаем среднее время, которое игрок проводит на каждом уровне.
Определим, сколько уровней игрок может пройти за 1 час.

*Шаг 1:* Вероятность прохождения одного уровня

- Вероятность проигрыша: 75% (0.75).
- Вероятность паузы: 20% (0.2).
- Вероятность успешного прохождения: 1 - Вероятность проигрыша - Вероятность паузы.

*Шаг 2:* Среднее время на уровне

- Время прохождения уровня: 3 минуты.
- Время паузы (если применяется): Среднее значение от 10 до 30 секунд.

*Шаг 3:* Количество уровней за 1 час

-Рассчитаем, сколько уровней может пройти игрок за 1 час, учитывая вероятность успешного прохождения и время на уровне.

In [23]:
import random

In [24]:
def play_game():
    # Вероятность проигрыша
    loss_probability = 0.75

    # Счетчик уровней
    levels = 0
    # Счетчик времени
    total_time = 0

    while total_time < 60:  # Пока не прошло 1 часа
        if random.uniform(0, 1) > loss_probability:  # Игрок выигрывает
            levels += 1
            total_time += 3  # Время на уровень

    return levels

In [25]:
# Запускаем игру много раз и усредняем результат
num_simulations = 10000
total_levels = sum(play_game() for _ in range(num_simulations))
average_levels = total_levels / num_simulations


In [26]:
print(f"Игрок вероятнее всего пройдет {average_levels:.2f} уровней за 1 час.")

Игрок вероятнее всего пройдет 20.00 уровней за 1 час.


 <a id='section4'></a>

### **4.** Назовите ТОП 5 мобильных игр и кратко опишите основные механики этих игр.

**Genshin Impact:** Экшен-RPG с открытым миром, где игроки исследуют Тиват, сражаются с монстрами, решают головоломки и прокачивают своих персонажей.

**PUBG Mobile:** Боевая рояльная игра. Игроки парашютируются на остров, собирают оружие и снаряжение, сражаются до последнего выжившего.

**Call of Duty: Mobile:** Мобильная версия популярной серии шутеров от первого лица. Игроки сражаются в различных режимах, используя разнообразное оружие.

**Fortnite:** Боевая рояльная игра с элементами строительства. Игроки сражаются, строят структуры и выживают в постоянно сжимающемся круге.

**Roblox:** Платформа с различными играми, созданными пользователями. Игроки могут создавать свои виртуальные миры или играть в чужие.